In [1]:
import ee
import geemap
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#ee.Authenticate()

In [2]:
# 获取matplotlib的色图
colormap = plt.cm.viridis  # 选择viridis色图

# 将色图转换为RGB颜色列表
def colormap_to_palette(cmap, n=256):
    """将matplotlib colormap转换为调色板"""
    colors = [cmap(i) for i in np.linspace(0, 1, n)]
    return ['#%02x%02x%02x' % (int(c[0] * 255), int(c[1] * 255), int(c[2] * 255)) for c in colors]

# 将色图转换为色带
palette = colormap_to_palette(colormap, n=256)  # 生成256个颜色

In [3]:
# 定义一个函数，将影像中的NDVI乘以0.0001
def scale_ndvi(image):
    return image.multiply(0.0001).copyProperties(image, image.propertyNames())

In [4]:
geemap.set_proxy(port=33210)
ee.Initialize()
map1 = geemap.Map()
# 加载MOD13Q1 NDVI数据
ndvi_collection = ee.ImageCollection("MODIS/061/MOD13Q1").select("NDVI").map(scale_ndvi)
vis = {
 'min': 0.0,
 'max': 1.0,
 'palette': 'viridis',
}
map1.addLayer(ndvi_collection, vis, 'NDVI')
#map1.add_time_slider(ndvi_collection, vis, time_interval=1) #时间动态
map1

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [5]:
type(ndvi_collection)

ee.imagecollection.ImageCollection

In [6]:
ndvi_collection.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'version': 1735068735245092,
 'id': 'MODIS/061/MOD13Q1',
 'properties': {'system:is_global': 1},
 'features': [{'type': 'Image',
   'bands': [{'id': 'NDVI',
     'data_type': {'type': 'PixelType',
      'precision': 'double',
      'min': -3.2768,
      'max': 3.2767},
     'dimensions': [172800, 72000],
     'crs': 'SR-ORG:6974',
     'crs_transform': [231.65635826395825,
      0,
      -20015109.354,
      0,
      -231.65635826395834,
      10007554.677003]}],
   'version': 1645924065142414,
   'id': 'MODIS/061/MOD13Q1/2000_02_18',
   'properties': {'system:asset_size': 29310900574,
    'system:index': '2000_02_18',
    'system:time_start': 950832000000,
    'google:max_source_file_timestamp': 1628741089000,
    'system:footprint': {'type': 'LinearRing',
     'coordinates': [[-180, -90],
      [180, -90],
      [180, 90],
      [-180, 90],
      [-180, -90]]},
    'system:time_end': 952214400000}},
  {'type': 'Image',
   'bands': [{'id': 'N

In [7]:
#上传FeatureCollection
site_buffer = geemap.shp_to_ee('F:/phd2/03sitefilter/site_buffers.shp')
map1.addLayer(site_buffer, {}, 'site_buffer')
type(site_buffer)

ee.featurecollection.FeatureCollection

In [8]:
#用FeatureCollection提取ImageCollection时间序列
mean_ts = geemap.zonal_stats(
        in_value_raster=ndvi_collection,
        in_zone_vector=site_buffer,
        out_file_path=None,
        stat_type='MEAN',
        scale=500,
        return_fc=True
    )
type(mean_ts)

Computing statistics ...


ee.featurecollection.FeatureCollection

In [9]:
#FeatureCollection变dataframe
ndvi_buffer = geemap.ee_to_df(mean_ts)
type(ndvi_buffer)

pandas.core.frame.DataFrame

In [10]:
#算每个缓冲区的最大、最小和标准差
selected_cols = ndvi_buffer.loc[:, '2000_02_18_NDVI':'2024_12_02_NDVI']
ndvi_buffer['max_buffer'] = selected_cols.max(axis=1)
ndvi_buffer['min_buffer'] = selected_cols.min(axis=1)
ndvi_buffer['std_buffer'] = selected_cols.std(axis=1)

# 构建新的 DataFrame
ndvi_buffer1 = ndvi_buffer[['siteid', 'max_buffer', 'min_buffer', 'std_buffer']]
ndvi_buffer1

siteid  max_buffer  min_buffer  std_buffer
0    AR_SLu    0.690095    0.289367    0.093649
1    AT_Neu    0.802641    0.019463    0.258924
2    AU_ASM    0.514582    0.202742    0.058470
3    AU_Cpr    0.533759    0.201445    0.066415
4    AU_Cum    0.715076    0.399041    0.058973
..      ...         ...         ...         ...
165  US_xTR    0.872231    0.076876    0.256639
166  US_xUK    0.824746    0.004033    0.182018
167  US_xUN    0.881420    0.068793    0.285402
168  US_xWD    0.770103   -0.066656    0.265650
169  ZM_Mon    0.700573    0.261860    0.088238

[170 rows x 4 columns]

In [11]:
#提取站点的NDVI
ndvi_band = ndvi_collection.toBands()
ndvi_site = geemap.extract_values_to_points(in_fc = site_buffer, image = ndvi_band, out_fc=None)
ndvi_site1 = geemap.ee_to_df(ndvi_site)
type(ndvi_site1)

pandas.core.frame.DataFrame

In [12]:
#算每个站点的最大、最小和标准差
selected_cols_site = ndvi_site1.loc[:, '2000_02_18_NDVI':'2024_12_02_NDVI']
ndvi_site1['max_site'] = selected_cols_site.max(axis=1)
ndvi_site1['min_site'] = selected_cols_site.min(axis=1)
ndvi_site1['std_site'] = selected_cols_site.std(axis=1)

# 构建新的 DataFrame
ndvi_site2 = ndvi_site1[['siteid', 'max_site', 'min_site', 'std_site']]
ndvi_site2

siteid  max_site  min_site  std_site
0    AR_SLu    0.7550    0.2785  0.113477
1    AT_Neu    0.9830   -0.1324  0.339625
2    AU_ASM    0.5221    0.1637  0.061918
3    AU_Cpr    0.5597    0.2018  0.073889
4    AU_Cum    0.8704    0.2024  0.165015
..      ...       ...       ...       ...
165  US_xTR    0.9364   -0.0069  0.298779
166  US_xUK    0.8081   -0.0152  0.177503
167  US_xUN    0.9173    0.0274  0.302102
168  US_xWD    0.8890   -0.0596  0.275737
169  ZM_Mon    0.8297    0.2002  0.110360

[170 rows x 4 columns]

In [13]:
ndvi_band.bandNames().getInfo()

['2000_02_18_NDVI',
 '2000_03_05_NDVI',
 '2000_03_21_NDVI',
 '2000_04_06_NDVI',
 '2000_04_22_NDVI',
 '2000_05_08_NDVI',
 '2000_05_24_NDVI',
 '2000_06_09_NDVI',
 '2000_06_25_NDVI',
 '2000_07_11_NDVI',
 '2000_07_27_NDVI',
 '2000_08_12_NDVI',
 '2000_08_28_NDVI',
 '2000_09_13_NDVI',
 '2000_09_29_NDVI',
 '2000_10_15_NDVI',
 '2000_10_31_NDVI',
 '2000_11_16_NDVI',
 '2000_12_02_NDVI',
 '2000_12_18_NDVI',
 '2001_01_01_NDVI',
 '2001_01_17_NDVI',
 '2001_02_02_NDVI',
 '2001_02_18_NDVI',
 '2001_03_06_NDVI',
 '2001_03_22_NDVI',
 '2001_04_07_NDVI',
 '2001_04_23_NDVI',
 '2001_05_09_NDVI',
 '2001_05_25_NDVI',
 '2001_06_10_NDVI',
 '2001_06_26_NDVI',
 '2001_07_12_NDVI',
 '2001_07_28_NDVI',
 '2001_08_13_NDVI',
 '2001_08_29_NDVI',
 '2001_09_14_NDVI',
 '2001_09_30_NDVI',
 '2001_10_16_NDVI',
 '2001_11_01_NDVI',
 '2001_11_17_NDVI',
 '2001_12_03_NDVI',
 '2001_12_19_NDVI',
 '2002_01_01_NDVI',
 '2002_01_17_NDVI',
 '2002_02_02_NDVI',
 '2002_02_18_NDVI',
 '2002_03_06_NDVI',
 '2002_03_22_NDVI',
 '2002_04_07_NDVI',


In [14]:
#检查是否存在全部为nan的行
#pd.set_option('display.max_rows', None)
all_nan_rows = ndvi_site2.isna().all(axis=1).sum()
all_nan_rows

0

In [15]:
ndvi_com = pd.merge(ndvi_buffer1, ndvi_site2, on='siteid', how='inner')
ndvi_com

siteid  max_buffer  min_buffer  std_buffer  max_site  min_site  std_site
0    AR_SLu    0.690095    0.289367    0.093649    0.7550    0.2785  0.113477
1    AT_Neu    0.802641    0.019463    0.258924    0.9830   -0.1324  0.339625
2    AU_ASM    0.514582    0.202742    0.058470    0.5221    0.1637  0.061918
3    AU_Cpr    0.533759    0.201445    0.066415    0.5597    0.2018  0.073889
4    AU_Cum    0.715076    0.399041    0.058973    0.8704    0.2024  0.165015
..      ...         ...         ...         ...       ...       ...       ...
165  US_xTR    0.872231    0.076876    0.256639    0.9364   -0.0069  0.298779
166  US_xUK    0.824746    0.004033    0.182018    0.8081   -0.0152  0.177503
167  US_xUN    0.881420    0.068793    0.285402    0.9173    0.0274  0.302102
168  US_xWD    0.770103   -0.066656    0.265650    0.8890   -0.0596  0.275737
169  ZM_Mon    0.700573    0.261860    0.088238    0.8297    0.2002  0.110360

[170 rows x 7 columns]

In [16]:
ndvi_com['max_bool'] = np.abs(ndvi_com['max_buffer'] - ndvi_com['max_site'])<=(0.2*ndvi_com['max_buffer'])
ndvi_com['min_bool'] = np.abs(ndvi_com['min_buffer'] - ndvi_com['min_site'])<=(0.2*ndvi_com['min_buffer'])
ndvi_com['std_bool'] = np.abs(ndvi_com['std_buffer'] - ndvi_com['std_site'])<=(0.2*ndvi_com['std_buffer'])
ndvi_com['site_max'] = ndvi_com['max_site']>=0.2
ndvi_com

siteid  max_buffer  min_buffer  std_buffer  max_site  min_site  std_site  \
0    AR_SLu    0.690095    0.289367    0.093649    0.7550    0.2785  0.113477   
1    AT_Neu    0.802641    0.019463    0.258924    0.9830   -0.1324  0.339625   
2    AU_ASM    0.514582    0.202742    0.058470    0.5221    0.1637  0.061918   
3    AU_Cpr    0.533759    0.201445    0.066415    0.5597    0.2018  0.073889   
4    AU_Cum    0.715076    0.399041    0.058973    0.8704    0.2024  0.165015   
..      ...         ...         ...         ...       ...       ...       ...   
165  US_xTR    0.872231    0.076876    0.256639    0.9364   -0.0069  0.298779   
166  US_xUK    0.824746    0.004033    0.182018    0.8081   -0.0152  0.177503   
167  US_xUN    0.881420    0.068793    0.285402    0.9173    0.0274  0.302102   
168  US_xWD    0.770103   -0.066656    0.265650    0.8890   -0.0596  0.275737   
169  ZM_Mon    0.700573    0.261860    0.088238    0.8297    0.2002  0.110360   

     max_bool  min_bool  std_bool  site_max  
0        True      True     False      True  
1       False     False     False      True  
2        True      True      True      True  
3        True      True      True      True  
4       False     False     False      True  
..        ...       ...       ...       ...  
165      True     False      True      True  
166      True     False      True      True  
167      True     False      True      True  
168      True     False      True      True  
169      True     False     False      True  

[170 rows x 11 columns]

In [17]:
ndvi_com.to_excel('F:/phd2/03sitefilter/ndvi_filter.xlsx')